In [1]:
from utils.dataloader import *
from archive.sift import *
from utils.helper import *
from utils.classifier import *
from sklearn.model_selection import train_test_split
from utils.features import *
import pickle

In [2]:
RANDOM_STATE = 33
ROOT_PATH, DATA_PATH, OUT_PATH = get_paths()
print(ROOT_PATH)

/Users/moooooo16/Documents/Computer Vision/Project/elpv


In [3]:
img_path, prob, types, labels= load_data(DATA_PATH)
print(len(img_path), 
      len(prob),
      len(types),
      len(labels))

[0 1 2 3 4 5 6 7] [588 117  56 313 920 178  50 402]
2624 2624 2624 2624


In [4]:
feature_builder = FeatureExtraction(DATA_PATH, img_path, labels)
X_train, X_test, y_train, y_test = feature_builder.split_data(train_test_split, randome_state=RANDOM_STATE, stratify=True, split_ratio=0.25)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1968, 300, 300) (1968,)
(656, 300, 300) (656,)


In [5]:
X_train, y_train = feature_builder.augmentation(X_train, y_train, augment_funcs=[flip_x, flip_y, rotate])
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

Augmenting images:   0%|          | 0/1968 [00:00<?, ?it/s]

Augmenting images: 100%|██████████| 1968/1968 [00:00<00:00, 85931.32it/s]

Augmenting done, added image: 903
(2871, 300, 300) (2871,)
(656, 300, 300) (656,)


In [6]:
X_train = feature_builder.preprocess(X_train, preprocess_funcs=[clach_img, guassian_blur, lap_feature])
X_test = feature_builder.preprocess(X_test, preprocess_funcs=[clach_img, guassian_blur, lap_feature])
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

Pre-processing images: 100%|██████████| 656/656 [00:00<00:00, 2123.33it/s]

(2871, 300, 300) (2871,)
(656, 300, 300) (656,)


In [7]:
X_train = feature_builder.get_hog_features(X_train, orient=9, pix_per_cell=10, vis = False)
X_test = feature_builder.get_hog_features(X_test, orient=9, pix_per_cell=10,vis = False)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

Calculating descriptors: 100%|██████████| 656/656 [00:05<00:00, 121.63it/s]


(2871, 30276) (2871,)
(656, 30276) (656,)


In [8]:
from sklearn.preprocessing import StandardScaler


X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2871, 30276) (2871,)
(656, 30276) (656,)


In [9]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.9)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2871, 1782) (2871,)
(656, 1782) (656,)


In [10]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


param = {
    'kernel': ['linear', 'rbf'],
    'C' : [0.01, 0.1, 1, 10, 100],
    'class_weight': ['balanced']
}


clf_collections = []
predictions_collections = []

In [11]:
clf = SVC()

best_clf = grid_search(X_train, y_train,clf, params= param, scoring = 'accuracy', classes = None, logger = None, k=None,verbose=4)
pred = best_clf.predict(X_test)

acc, conf_mat = get_report(y_test, pred)
print()
print(f'Overall Test Accuracy: {acc:.2f}')
print(f'Confusion Matrix: \n{conf_mat}')
print('-'*100)

clf_collections.append(best_clf)
predictions_collections.append(pred)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 2/5] END C=0.01, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.570) total time=  21.7s
[CV 1/5] END C=0.01, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.503) total time=  21.9s
[CV 4/5] END C=0.01, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.605) total time=  22.6s
[CV 3/5] END C=0.01, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.611) total time=  22.5s
[CV 5/5] END C=0.01, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.601) total time=  22.8s
[CV 2/5] END C=0.01, class_weight=balanced, kernel=rbf;, score=(train=0.154, test=0.153) total time=  30.9s
[CV 1/5] END C=0.01, class_weight=balanced, kernel=rbf;, score=(train=0.082, test=0.082) total time=  31.2s
[CV 3/5] END C=0.01, class_weight=balanced, kernel=rbf;, score=(train=0.154, test=0.153) total time=  31.4s
[CV 1/5] END C=0.1, class_weight=balanced, kernel=linear;, s

In [11]:
clf = SVC

total_classes = np.unique(y_train)
best_clfs = one_vs_other_up_sampling(X_train = X_train, y_train=y_train, 
                                     total_classes= total_classes,
                                     clf = SVC, param = param, 
                                     k = None, logger = None, knn= KNeighborsClassifier, n_neighbors=5, verbose=4)

distance , pred = distance_vote(X_test, best_clfs, None)

acc, conf_mat = get_report(y_test, pred)

print()
print(f'Overall Test Accuracy: {acc:.2f}')
print(f'Confusion Matrix: \n{conf_mat}')
print('-'*100)

clf_collections.append(best_clfs)
predictions_collections.append(pred)


Generated 2205 synthetic samples.
Trainig on: (array([-1.,  0.]), array([2430, 2646]))
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END C=0.01, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.944) total time=  12.8s
[CV 4/5] END C=0.01, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.945) total time=  13.2s
[CV 2/5] END C=0.01, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.963) total time=  13.6s
[CV 3/5] END C=0.01, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.967) total time=  13.8s
[CV 5/5] END C=0.01, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.953) total time=  14.1s
[CV 1/5] END C=0.1, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.944) total time=  11.8s
[CV 2/5] END C=0.1, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.963) total time=  12.3s
[CV 3/5] END C=0.1, class_weight=balanced, kernel=linear;, score=(train=1.000

/opt/homebrew/Caskroom/miniconda/base/envs/torch/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 4/5] END C=0.01, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.972) total time=  14.9s
[CV 3/5] END C=0.01, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.977) total time=  15.4s
[CV 1/5] END C=0.1, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.924) total time=  13.6s
[CV 2/5] END C=0.1, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.970) total time=  14.6s
[CV 3/5] END C=0.1, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.977) total time=  15.0s
[CV 4/5] END C=0.1, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.972) total time=  12.4s
[CV 5/5] END C=0.1, class_weight=balanced, kernel=linear;, score=(train=1.000, test=0.980) total time=  12.6s
[CV 2/5] END C=0.01, class_weight=balanced, kernel=rbf;, score=(train=0.659, test=0.622) total time=  54.4s
[CV 3/5] END C=0.01, class_weight=balanced, kernel=rbf;, score=(train=0.669, test=0.684) total time=  54.8s
[CV 1/5] END

NameError: name 'best_clf' is not defined